# Visualization Multi Objective, N-dimensional

## Imports

In [ ]:
# Enable autoreload
%load_ext autoreload
%autoreload 2

### Packages

In [ ]:
import os
import ast
import json
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import ScalarFormatter, LogLocator, FuncFormatter, MaxNLocator

In [ ]:
from src.plots import plot_param_relationships, plot_3d_results_with_target_range, plot_3d_in_out_interactive
# from src.h

### Input 

In [ ]:
# log_name = 'LoanApp_SR_S1'
# log_name = 'LoanApp_SR_S2'
# log_name = 'LoanApp_SR_S3'
log_name = 'LoanApp_SR_S4_3'

if log_name == 'LoanApp_SR_S1':
    experiment_name = '20251125_230351_2D_1PPIs_30n_0.9a_0.85b_paper_S1'

if log_name == 'LoanApp_SR_S2':
    experiment_name = '20251125_230354_2D_1PPIs_30n_0.9a_0.85b_paper_S2'

if log_name == 'LoanApp_SR_S3':
    experiment_name = '20251125_230513_2D_2PPIs_30n_0.9a_0.85b_paper_S3'

if log_name == 'LoanApp_SR_S4_3':
    experiment_name = '20251125_230559_3D_2PPIs_30n_0.9a_0.75b_paper_S4'
    

if log_name == 'LoanApp_SingleRole':
    experiment_name = '20251115_183049_3D_2PPIs_10n_0.95a_0.9b'

if log_name == 'LoanApp_TwoRoles':
    experiment_name = '20251112_141621'

if log_name == 'LoanApp_SingleRole_fixDuration':
    experiment_name = '20251114_142758' 

if log_name == 'ten_activities' :
    experiment_name = '20251107_175045_onlyCost'



data_dir = os.path.join('data')


### Data

In [ ]:
# experiment_path = os.path.join(data_dir, log_name, 'output', experiment_name)
output_dir = os.path.join('data', log_name, 'output', experiment_name)
output_figures_path = os.path.join(output_dir, 'figures')
results_df_path = os.path.join(output_dir, 'simulation_log.csv')
params_path     = os.path.join(output_dir, 'params.json')

In [ ]:
# Create output directories for figures
os.makedirs(output_figures_path, exist_ok=True)

In [ ]:
# read in params
with open(params_path, 'r') as f:
    params = json.load(f) 

In [ ]:
# read in results dataframe
results_df = pd.read_csv(results_df_path)

In [ ]:
# convert columns with string representations of dicts/lists back to actual dicts/lists
dict_columns = ['target_range', 'target_ppi_means', 'target_ppi_dict', 'target_ppis']
for col in dict_columns:
    results_df[col] = results_df[col].apply(ast.literal_eval)

In [ ]:

# sampled_points_path = params['strategies']['quadtree']['paths']['sampled_points']
sampled_points_path = os.path.join(output_dir, 'hyperquadtree_sampled_points.csv')
hqt_df = pd.read_csv(sampled_points_path)

In [ ]:
params

In [ ]:
params['viz'] = {
    # 'x_param': None, # 'arriaval_distr_mean',
    # 'y_param': None, # 'resource_count_UnifiedResourceProfile',
    # 'z_col':  None, # params['target_ppi'],
    'output_figures_path': output_figures_path,
    'figure_extensions': ['.png', '.eps', '.pdf'],
    'plot_2d': True,
    'plot_3d_ppi': True,
    'plot_3d_InOut': False,
    '2d_params': {
        'x_param': None,
        'y_param': None,
        'target_ppi': None,
    },
    '3d_ppi_params': {
        'x_param': None,    
        'y_param': None,
        'z_col': None,
    },
    '3d_InOut_params': {
        'x_param': None,    
        'y_param': None,
        'z_param': None,
    }
}

In [ ]:
params_to_change = list(params['params_to_change'].keys())
target_ppis = params['target_ppis']

if len(params_to_change) == 2:

    params['viz']['2d_params']['x_param'] = params_to_change[0]
    params['viz']['2d_params']['y_param'] = params_to_change[1]
    params['viz']['plot_3d_ppi'] = True

    if len(target_ppis) == 1:
        params['viz']['3d_ppi_params']['x_param'] = params_to_change[0]
        params['viz']['3d_ppi_params']['z_param'] = params_to_change[1]
        params['viz']['3d_ppi_params']['y_col'] = target_ppis
    
    else:
        params['viz']['plot_3d_ppi'] = False

if len(params_to_change) != 2: 
    params['viz']['plot_2d'] = False
    params['viz']['plot_3d_ppi'] = False

if len(params_to_change) == 3: 
    params['viz']['plot_3d_InOut'] = True
    params['viz']['3d_InOut_params'] = {
        'x_param': params_to_change[0],
        'y_param': params_to_change[1],
        'z_param': params_to_change[2],
    }

if len(params_to_change) == 3: 
    for param in params_to_change:

        if 'resource_count' in param:
            params['viz']['3d_InOut_params']['x_param'] = param
        if 'arriaval_distr_mean' in param:
            params['viz']['3d_InOut_params']['y_param'] = param
        if 'branching_probability' in param:
            params['viz']['3d_InOut_params']['z_param'] = param

print(params['viz']['plot_3d_ppi'])


In [ ]:
plot_vars = { 
    'label_size': 34,                      # Size of axis labels
    'tick_size': 24,                       # Size of tick labels
    'figure_size': (8, 8),                 # Reduced height for the figure
    'tree_node_colour': 'blue',        # Color of the tree nodes
    'plot_title': False,                      # Whether to display the plot title
    'plot_legend': False,                     # Whether to display the legend
    'legend_location': 'best',               # Location of the legend
    'legend_size': 12,                      # Font size of the legend
    'line_width': 1.5,                     # Line width for the edges of the bars
    'transparancy': 0.375,                        # Transparency of the bars
    'grid_alpha': 0.3,                     # Transparency of the grid


    # 'x_label': 'Avg. lead times (hours)',  # Label for the x-axis
    # 'y_label': 'Frequency',                # Label for the y-axis
    # 'vertical_line_color': 'blue',         # Color of the vertical line
    # 'vertical_line_width': 4               # Thickness of the vertical line
    'label_dict': {
        'resource_count_': 'Number of resources',
        # 'arriaval_distr_mean': 'Arrival distribution mean (m)',
        'arriaval_distr_mean': 'Arrival times (m)',
        'branching_probability': 'Branching probability',
        'cost': 'Cost',
        'cycle_time': 'Cycle Time',
        'lead_time': 'Lead time',
    }
}
params['viz']['plot_vars'] = plot_vars



## Preparation

### Dataframe prep

#### Removal of unnecessary params

In [ ]:
# Drop the columns 'ppi_value' and 'distance' if they exist in the DataFrame
columns_to_drop = ['ppi_value', 'distance']
results_df = results_df.drop(columns=[col for col in columns_to_drop if col in results_df.columns])


#### Backwards comparability creation

In [ ]:
# Version connectivity of naming
results_df = results_df.rename(columns={'target_ppi_mean': 'target_ppi_val'})

# if the column status is present, replace True with "in" and False with "out"
if 'status' in results_df.columns:
    results_df['status'] = results_df['status'].replace({True: 'in', False: 'out'})

## Visulaize

### Prep

In [ ]:
algorithms = results_df['algorithm'].unique().tolist()

In [ ]:

def get_label(param_name, label_dict):
    for key in label_dict.keys():
        if param_name.startswith(key):
            label = label_dict[key]
            return label


### Plot

### 2D

In [ ]:
# from src.plots import plot_gs_hqt_overlay

if params['viz']['plot_2d']:
    algorithms_list = list(algorithms)
    print(algorithms_list)
    always_include = ['grid_search', 'orig_run']
    algorithms_list = [alg for alg in algorithms if alg not in always_include]
    algorithms_list

    if algorithms_list == []:
        algorithms_list = ['grid_search']

    for algorithm in algorithms_list:

        print(f"\n ### Generating 3D plot for algorithm: {algorithm}")
        # Create a copy of the dataframe
        filtered_df = results_df.copy()

        algorithms_to_plot = always_include.copy()
        algorithms_to_plot.append(algorithm)

        # Filter the dataframe for 'grid_search' and one other algorithm (e.g., 'hill_descent_random_one')
        filtered_df = filtered_df[filtered_df['algorithm'].isin(algorithms_to_plot)]

        target_range = params['target_range']
        plot_gs_hqt_overlay_with_areas(results_df, params, hqt_df=hqt_df)#(filtered_df, algorithms=algorithms, params=params)




In [ ]:
# from src.plots import plot_param_relationships

if params['viz']['plot_2d']:
    algorithms_list = list(algorithms)
    print(algorithms_list)
    always_include = ['grid_search', 'orig_run']
    algorithms_list = [alg for alg in algorithms if alg not in always_include]
    algorithms_list

    if algorithms_list == []:
        algorithms_list = ['grid_search']

    for algorithm in algorithms_list:

        print(f"\n ### Generating 3D plot for algorithm: {algorithm}")
        # Create a copy of the dataframe
        filtered_df = results_df.copy()

        algorithms_to_plot = always_include.copy()
        algorithms_to_plot.append(algorithm)

        # Filter the dataframe for 'grid_search' and one other algorithm (e.g., 'hill_descent_random_one')
        filtered_df = filtered_df[filtered_df['algorithm'].isin(algorithms_to_plot)]

        target_range = params['target_range']
        plot_param_relationships(filtered_df, algorithms=algorithms, params=params)




### 3D

In [ ]:


if params['viz']['plot_3d_ppi']:

    for algorithm in algorithms:
        if algorithm != 'orig_run':
                
            print(f"\n ### Generating scatter plots for algorithm: {algorithm}") 

            # Define the columns for the axes
            x_col = params['viz']['3d_ppi_params']['x_param']
            z_col = params['viz']['3d_ppi_params']['z_param']
            
            
            y_col_values = params['viz']['3d_ppi_params']['y_col']
            for y_col in y_col_values:
                print(f"Plotting for y_col: {y_col}")

                # Call the function to plot the 3D graph
                upperbound = params['target_range'][y_col][1]
                y_max_value = upperbound * 2.5
                plot_3d_results_with_target_range(results_df, params, algorithm,  x_col, z_col, y_col, params['target_range'][y_col], use_log_scale=True, draw_plane=False)
                plot_3d_results_with_target_range(results_df, params, algorithm,  x_col, z_col, y_col, params['target_range'][y_col], use_log_scale=False, draw_plane=False, y_max_value=y_max_value)
                # plot_3d_results_with_target_range(results_df, algorithm, x_col, z_col, y_col, params['target_range'], use_log_scale=False, draw_plane=False)

In [ ]:
from src.plots import plot_3d_hqt_leaf_nodes

if params['viz']['plot_3d_InOut']:
    plot_3d_hqt_leaf_nodes(hqt_df, params)

### 3D 3 Params

In [ ]:
from src.plots import plot_param_relationships, plot_3d_results_with_target_range, plot_3d_in_out, plot_3d_in_out_interactive
if params['viz']['plot_3d_InOut']:

    for algorithm in algorithms:
        if algorithm != 'orig_run':
                
            print(f"\n ### Generating 3D In/Out plots for algorithm: {algorithm}") 
            
            # Call the function to plot the 3D graph
            angles=(5, 85)
            angles=(0, 90)
            # angles=(45, 45)

            # for area in ['in', 'out']:
            for area in ['in']:
                
                plot_3d_in_out(results_df, 
                           params, 
                           algorithm, 
                           use_connected_surface=False, 
                           angles=angles, 
                           show_points=area)